In [1]:
import pandas as pd
import numpy as np
import chardet
import matplotlib.pyplot as plt
import seaborn as sns
import base64
from io import BytesIO

### Data loading with specfic encoding

In [2]:
drink_path = './../dataset/starbucks-menu-nutrition-drinks.csv'
food_path = './../dataset/starbucks-menu-nutrition-food.csv'

with open(food_path, 'rb') as f:
    result = chardet.detect(f.read())
    print(f'Food: {result}')

with open(drink_path, 'rb') as f:
    result = chardet.detect(f.read())
    print(f'Drink: {result}')



Food: {'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}
Drink: {'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [3]:
food = pd.read_csv(food_path, encoding='UTF-16')
drink = pd.read_csv(drink_path, encoding='UTF-8')

### Basic EDA for Food Menu

In [4]:
food.head()

,Unnamed: 0,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
0,Chonga Bagel,300,5.0,50,3,12
1,8-Grain Roll,380,6.0,70,7,10
2,Almond Croissant,410,22.0,45,3,10
3,Apple Fritter,460,23.0,56,2,7
4,Banana Nut Bread,420,22.0,52,2,6


### First column name + Noticed empty whitespaces in front of column names

In [5]:
food.columns

Index(['Unnamed: 0', ' Calories', ' Fat (g)', ' Carb. (g)', ' Fiber (g)',
       ' Protein (g)'],
      dtype='object')

In [6]:
food.rename(columns={food.columns[0]: 'Item'}, inplace=True)
food.columns = [name.strip() for name in food.columns]

In [7]:
food.columns

Index(['Item', 'Calories', 'Fat (g)', 'Carb. (g)', 'Fiber (g)', 'Protein (g)'], dtype='object')

### Consistent Dtype across all rows and columns (113 row all with proper data - int/float) for Food menu

In [8]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Item         113 non-null    object 
 1   Calories     113 non-null    int64  
 2   Fat (g)      113 non-null    float64
 3   Carb. (g)    113 non-null    int64  
 4   Fiber (g)    113 non-null    int64  
 5   Protein (g)  113 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 5.4+ KB


In [9]:
food.isna().sum()   

Item           0
Calories       0
Fat (g)        0
Carb. (g)      0
Fiber (g)      0
Protein (g)    0
dtype: int64

### Basic description of Food menu (Averages, standard deviations, min/max, 25, 50 and 75 percentile)

In [10]:
food.describe()

,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
count,113.000000,113.000000,113.000000,113.000000,113.000000
mean,356.637168,16.353982,41.486726,2.849558,11.469027
std,127.710685,8.297397,15.796764,2.888466,8.463230
min,90.000000,0.000000,5.000000,0.000000,1.000000
25%,280.000000,9.000000,31.000000,1.000000,5.000000
50%,360.000000,17.000000,42.000000,2.000000,8.000000
75%,450.000000,23.000000,53.000000,3.000000,19.000000
max,650.000000,37.000000,80.000000,21.000000,34.000000


In [11]:
food

,Item,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
0,Chonga Bagel,300,5.0,50,3,12
1,8-Grain Roll,380,6.0,70,7,10
2,Almond Croissant,410,22.0,45,3,10
3,Apple Fritter,460,23.0,56,2,7
4,Banana Nut Bread,420,22.0,52,2,6
...,...,...,...,...,...,...
108,Justin's Chocolate Hazelnut Butter,180,14.0,12,3,4
109,Justin's Classic Almond Butter,190,18.0,6,3,7
110,Lemon Chiffon Yogurt,340,13.0,38,0,18
111,Organic Avocado (Spread),90,8.0,5,4,1


In [12]:
drink.columns

Index(['Unnamed: 0', 'Calories', 'Fat (g)', 'Carb. (g)', 'Fiber (g)',
       'Protein', 'Sodium'],
      dtype='object')

In [13]:
food.columns

Index(['Item', 'Calories', 'Fat (g)', 'Carb. (g)', 'Fiber (g)', 'Protein (g)'], dtype='object')

### Basic EDA for Drink Menu

In [14]:
drink.head()

,Unnamed: 0,Calories,Fat (g),Carb. (g),Fiber (g),Protein,Sodium
0,Cool Lime Starbucks Refreshers™ Beverage,45,0,11,0,0,10
1,Ombré Pink Drink,-,-,-,-,-,-
2,Pink Drink,-,-,-,-,-,-
3,Strawberry Acai Starbucks Refreshers™ Beverage,80,0,18,1,0,10
4,Very Berry Hibiscus Starbucks Refreshers™ Beve...,60,0,14,1,0,10


In [15]:
drink.rename(columns={drink.columns[0]: 'Item'}, inplace=True)

In [16]:
drink.columns

Index(['Item', 'Calories', 'Fat (g)', 'Carb. (g)', 'Fiber (g)', 'Protein',
       'Sodium'],
      dtype='object')

### Incorrect/Inconsistent Columns
1. Incorrect Dtype across columns (object data types present in columns 1 to 6 
-> supposed to be int/float) for Drink menu

2. Inconsistent Column naming (units missing for Protein and Sodium)
--> Remove units for all columns to standarise 
(Assumption made: Unit=grams for all columns other than Calories)


In [17]:
drink.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Item       177 non-null    object
 1   Calories   177 non-null    object
 2   Fat (g)    177 non-null    object
 3   Carb. (g)  177 non-null    object
 4   Fiber (g)  177 non-null    object
 5   Protein    177 non-null    object
 6   Sodium     177 non-null    object
dtypes: object(7)
memory usage: 9.8+ KB


### Data preprocessing function for csv files 

In [18]:
def clean_df(df):
    """
    This function cleans the dataframe by:
    1. Rename the first column to 'Item'
    2. Strip leading/trailing whitespaces from column names
    3. Remove units from column names
    4. Converting all columns to numeric
    5. Removing rows with NaN values
    6. Removing duplicate rows (if any)
    7. Removing rows with all 0 values
    """
    df.rename(columns={df.columns[0]: 'Item'}, inplace=True) #Rename the first column to 'Item'
    df.columns = [name.strip().split(' ')[0] for name in df.columns] #Strip leading/trailing whitespaces from column names + Remove units from column names
    numeric = df.iloc[:, 1:].apply(lambda x: pd.to_numeric(x, errors='coerce')) #Convert non-item columns to numeric (coerce errors to NaN)
    df = pd.concat([df['Item'], numeric], axis=1) #Concatenate the 'Item' column with the numeric columns
    df.dropna(inplace=True) 
    df.drop_duplicates(inplace=True)
    df = df[[not all(row == 0) for row in df.iloc[:, 1:].values]] #Remove rows with all 0 values

    return df

In [19]:
drink = clean_df(drink)

In [20]:
drink.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 0 to 173
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Item      73 non-null     object 
 1   Calories  73 non-null     float64
 2   Fat       73 non-null     float64
 3   Carb.     73 non-null     float64
 4   Fiber     73 non-null     float64
 5   Protein   73 non-null     float64
 6   Sodium    73 non-null     float64
dtypes: float64(6), object(1)
memory usage: 4.6+ KB


In [21]:
drink.describe()

,Calories,Fat,Carb.,Fiber,Protein,Sodium
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
mean,140.616438,2.746575,24.589041,0.561644,4.863014,65.753425
std,101.584479,4.182572,15.994648,1.572209,5.996098,72.147954
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60.000000,0.000000,13.000000,0.000000,0.000000,10.000000
50%,130.000000,0.000000,24.000000,0.000000,1.000000,15.000000
75%,200.000000,5.000000,35.000000,0.000000,10.000000,135.000000
max,430.000000,26.000000,64.000000,8.000000,20.000000,240.000000
